**<font color="blue">HW 5 – Multi-label Classification Using Hugging Face Train**</font>

In [1]:
# If in Colab, then import the drive module from google.colab
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  # Mount the Google Drive to access files stored there
  drive.mount('/content/drive')

  # Install the latest version of torchtext library quietly without showing output

  !pip install transformers evaluate wandb datasets accelerate  -U -qq  ## NEW LINES ##

  basepath = '/content/drive/MyDrive/Applied NLP/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:0

**<font color="blue">Import the necessary libraries</font>**

In [2]:
# Importing PyTorch library for tensor computations and neural network modules
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from pathlib import Path
from sklearn.metrics import confusion_matrix
from datetime import datetime

# For plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
# %matplotlib inline

### NEW ##########################
# imports from Huggingface ecosystem
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import PreTrainedModel, PretrainedConfig
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import evaluate

# wandb library
import wandb

In [3]:
base_folder = Path(basepath)
data_folder = base_folder/'data'
model_folder = base_folder/'models'
custom_functions = base_folder/'Codes/Custom py classes'

In [4]:
model_folder.mkdir(exist_ok=True, parents = True)

In [5]:
model_folder

PosixPath('/content/drive/MyDrive/Applied NLP/models')

In [6]:
# check the models we have dowloaded in spacy folder
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


**<font color="blue">Read Training and Test data**</font>

In [7]:
traindata = pd.read_csv(data_folder/'train.csv')
testdata = pd.read_csv(data_folder/'test.csv')

In [8]:
traindata.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2017-21441,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,2017-31535,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,2017-21068,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,2017-31436,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,2017-22195,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


**<font color="blue">Store the output labels</font>**

In [9]:
# Assuming your dataframe is called `df`
trainlabels = traindata[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
             'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].values
trainlabels

array([[0, 1, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       ...,
       [1, 0, 1, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

**<font color="blue">Check for class imbalance. By the output counts we can see that there is class imbalance so we will have to account for this during the training.</font>**

In [10]:

# Assuming your dataframe is named 'traindata'
num_ones_per_column = traindata[['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
             'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].sum()

print(num_ones_per_column)

anger           2859
anticipation    1102
disgust         2921
fear            1363
joy             2877
love             832
optimism        2291
pessimism        895
sadness         2273
surprise         396
trust            400
dtype: int64


**<font color="blue">Basic Cleaning of the input data</font>**

In [11]:
# We will load the model -en_core_web_sm
import spacy
nlp = spacy.load('en_core_web_sm')

In [12]:
# Import the List type from the typing module to use in function annotations
from typing import List

# for basic cleaning
from bs4 import BeautifulSoup
import re

from pprint import pprint
def basic_clean(text: str) -> str:
    """
    This function performs basic text cleaning on an input string by removing HTML tags (if present)
    and replacing newline and return characters with a space.

    Parameters:
    text (str): The input string to be cleaned

    Returns:
    str: The cleaned string
    """
    # Use BeautifulSoup to remove HTML tags (if present)
    soup = BeautifulSoup(text, "html.parser")
    text = soup.get_text()

    # Replace newline and return characters with a space
    return re.sub(r'[\n\r]',' ', text)

In [13]:
X_train_cleaned= [basic_clean(text).encode('utf-8', 'ignore').decode() for text in traindata['Tweet'].values]
X_test_cleaned= [basic_clean(text).encode('utf-8', 'ignore').decode() for text in testdata['Tweet'].values]

<ipython-input-12-6e593a7ba5cb>:21: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, "html.parser")


In [14]:
X_train_cleaned[0]

"“Worry is a down payment on a problem you may never have'. \xa0Joyce Meyer.  #motivation #leadership #worry"

In [15]:
X_test_cleaned[0]

'@Adnan__786__ @AsYouNotWish Dont worry Indian army is on its ways to dispatch all Terrorists to Hell'

**<font color="blue">Split traindata into training and validation sets</font>**

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_valid, trainlabels, validlabels = train_test_split(X_train_cleaned, trainlabels, test_size=0.2, random_state=42)

**<font color="blue">Preparing three datasets using Huggingface dataset library</font>**

In [17]:
trainset = Dataset.from_dict({
    'texts': X_train,
    'labels': trainlabels
})

validset = Dataset.from_dict({
    'texts': X_valid,
    'labels': validlabels
})
testset = Dataset.from_dict({
    'texts': X_test_cleaned
})

In [18]:
trainset

Dataset({
    features: ['texts', 'labels'],
    num_rows: 6179
})

**<font color="blue">Define a custom neural network model called CustomMLP that extends PreTrainedModel from the Hugging Face Transformers library.</font>**

In [19]:
from transformers import PretrainedConfig

class CustomConfig(PretrainedConfig):
    def __init__(self, vocab_size=0, embedding_dim=0, hidden_dim1=0, hidden_dim2=0, hidden_dim3=0, num_labels=2, dropout_rate=0.3, **kwargs):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim1 = hidden_dim1
        self.hidden_dim2 = hidden_dim2
        self.hidden_dim3 = hidden_dim3  # Added third hidden layer dimension
        self.num_labels = num_labels
        self.dropout_rate = dropout_rate  # Included dropout rate



In [20]:
from transformers.modeling_outputs import SequenceClassifierOutput
import torch
import torch.nn as nn

class CustomMLP(PreTrainedModel):
    config_class = CustomConfig

    def __init__(self, config):
        super().__init__(config)

        # Randomly initialized embeddings using the vocab size
        self.embedding_bag = nn.EmbeddingBag(config.vocab_size, config.embedding_dim)

        # LSTM layer after embedding
        self.lstm = nn.LSTM(
            input_size=config.embedding_dim,
            hidden_size=config.hidden_dim1,
            batch_first=True,
            bidirectional=True #using bidirectional LSTM, (since it's bidirectional, the output size is doubled
        )

        # Fully connected layers with BatchNorm, Dropout, and LeakyReLU
        self.fc = nn.Sequential(
            nn.Linear(config.hidden_dim1 * 2, config.hidden_dim2),  # *2 for bidirectional LSTM
            nn.BatchNorm1d(num_features=config.hidden_dim2), #  to normalize the output.
            nn.LeakyReLU(negative_slope=0.01), #introduce non-linearity with a small slope for negative values.
            nn.Dropout(p=config.dropout_rate),#to prevent overfitting.

            nn.Linear(config.hidden_dim2, config.hidden_dim3),
            nn.BatchNorm1d(num_features=config.hidden_dim3),
            nn.LeakyReLU(negative_slope=0.01),
            nn.Dropout(p=config.dropout_rate),

            nn.Linear(config.hidden_dim3, config.num_labels)
        )

    def forward(self, input_ids, offsets, labels=None):
        # Ensure all inputs are on the same device
        device = input_ids.device

        # Embedding layer with LSTM
        embed_out = self.embedding_bag(input_ids, offsets).unsqueeze(1)  # Adding batch dimension for LSTM
        lstm_out, _ = self.lstm(embed_out)
        lstm_out = lstm_out[:, -1, :]  # Take the last hidden state

        # Feed the LSTM output through the fully connected layers
        logits = self.fc(lstm_out)

        loss = None
        if labels is not None:
            # Ensure labels are on the same device
            labels = labels.float().to(device)
            # Dynamically compute pos_weight to account for class imbalance(here adding uniform weights to all the classes)
            pos_weight = torch.ones(self.config.num_labels).to(device) * 2.0
            #and ensure it's on the same device
            pos_weight = pos_weight.to(device)
             # Compute loss
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
            loss = loss_fct(logits, labels)

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits
        )


**<font color="blue">Creating Vocab for the tweets</font>**

In [21]:
from collections import Counter, OrderedDict
from typing import Dict, List, Optional, Union

class Vocab:
    def __init__(self, tokens: List[str]) -> None:
        self.itos: List[str] = tokens
        self.stoi: Dict[str, int] = {token: i for i, token in enumerate(tokens)}
        self.default_index: Optional[int] = None

    def __getitem__(self, token: str) -> int:
        if token in self.stoi:
            return self.stoi[token]
        if self.default_index is not None:
            return self.default_index
        raise RuntimeError(f"Token '{token}' not found in vocab")

    def __contains__(self, token: str) -> bool:
        return token in self.stoi

    def __len__(self) -> int:
        return len(self.itos)

    def insert_token(self, token: str, index: int) -> None:
        if index < 0 or index > len(self.itos):
            raise ValueError("Index out of range")
        if token in self.stoi:
            old_index = self.stoi[token]
            if old_index < index:
                self.itos.pop(old_index)
                self.itos.insert(index - 1, token)
            else:
                self.itos.pop(old_index)
                self.itos.insert(index, token)
        else:
            self.itos.insert(index, token)

        self.stoi = {token: i for i, token in enumerate(self.itos)}

    def append_token(self, token: str) -> None:
        if token in self.stoi:
            raise RuntimeError(f"Token '{token}' already exists in the vocab")
        self.insert_token(token, len(self.itos))

    def set_default_index(self, index: Optional[int]) -> None:
        self.default_index = index

    def get_default_index(self) -> Optional[int]:
        return self.default_index

    def lookup_token(self, index: int) -> str:
        if 0 <= index < len(self.itos):
            return self.itos[index]
        raise RuntimeError(f"Index {index} out of range")

    def lookup_tokens(self, indices: List[int]) -> List[str]:
        return [self.lookup_token(index) for index in indices]

    def lookup_indices(self, tokens: List[str]) -> List[int]:
        return [self[token] for token in tokens]

    def get_stoi(self) -> Dict[str, int]:
        return self.stoi.copy()

    def get_itos(self) -> List[str]:
        return self.itos.copy()

    @classmethod
    def vocab(cls, ordered_dict: Union[OrderedDict, Counter], min_freq: int = 1, specials: Optional[List[str]] = None, special_first: bool = True) -> 'Vocab':
        specials = specials or []
        for token in specials:
            ordered_dict.pop(token, None)

        tokens = [token for token, freq in ordered_dict.items() if freq >= min_freq]

        if special_first:
            tokens = specials + tokens
        else:
            tokens = tokens + specials

        return cls(tokens)

In [22]:
def get_vocab(dataset, min_freq=1):
    """
    Generate a vocabulary from a dataset.

    Args:
        dataset (Dataset): A Hugging Face Dataset object. The dataset should
                           have a key 'texts' that contains the text data.
        min_freq (int): The minimum frequency for a token to be included in
                        the vocabulary.

    Returns:
        torchtext.vocab.Vocab: Vocabulary object containing tokens from the
                               dataset that meet or exceed the specified
                               minimum frequency. It also includes a special
                               '<unk>' token for unknown words.
    """
    # Initialize a counter object to hold token frequencies
    counter = Counter()

    # Update the counter with tokens from each text in the dataset
    # Iterating through texts in the dataset
    for text in dataset['texts']:  ###### Change from previous function ####
        counter.update(str(text).split())

    # Create a vocabulary using the counter object
    # Tokens that appear fewer times than `min_freq` are excluded
    my_vocab = Vocab.vocab(counter, min_freq=min_freq)

    # Insert a '<unk>' token at index 0 to represent unknown words
    my_vocab.insert_token('<unk>', 0)

    # Set the default index to 0
    # This ensures that any unknown word will be mapped to '<unk>'
    my_vocab.set_default_index(0)

    return my_vocab

In [23]:
# Creating a function that will be used to get the indices of words from vocab
def tokenizer(text, vocab):
    """Converts text to a list of indices using a vocabulary dictionary"""
    return [vocab[token] for token in str(text).split()]

**<font color="blue">Collate Function</font>**

In [24]:
import torch

def collate_batch(batch, my_vocab, is_test=False):
    """
    Prepares a batch of data by transforming texts into indices based on a vocabulary and,
    optionally, converting labels into a tensor (if available).

    Args:
        batch (list of dict): A batch of data where each element is a dictionary with keys
                              'labels' (optional) and 'texts'. 'labels' are optional sentiment labels,
                              and 'texts' are the corresponding texts.
        my_vocab (torchtext.vocab.Vocab): A vocabulary object that maps tokens to indices.
        is_test (bool): If True, assumes that the batch is for testing and does not require labels.

    Returns:
        dict: A dictionary with the following keys:
              - 'input_ids': a tensor containing concatenated indices of the texts.
              - 'offsets': a tensor representing the starting index of each text in 'input_ids'.
              - 'labels': (optional) a tensor of the labels for each text in the batch, if labels are provided.
    """

    # Get texts from batch dict samples
    texts = [sample['texts'] for sample in batch]

    # Convert the list of texts into a list of lists; each inner list contains the vocabulary indices for a text
    list_of_list_of_indices = [tokenizer(text, my_vocab) for text in texts]

    # Concatenate all text indices into a single tensor
    input_ids = torch.cat([torch.tensor(i, dtype=torch.int64) for i in list_of_list_of_indices])

    # Compute the offsets for each text in the concatenated tensor
    offsets = [0] + [len(i) for i in list_of_list_of_indices]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)

    batch_dict = {
        'input_ids': input_ids,
        'offsets': offsets
    }

    # If it's not test data, include the labels in the output
    if not is_test:
        # Get labels from batch and convert them into a tensor of dtype int32
        labels = [sample['labels'] for sample in batch]
        labels = torch.tensor(labels, dtype=torch.long)
        batch_dict['labels'] = labels

    return batch_dict


In [25]:
tweet_vocab = get_vocab(trainset, min_freq=2)
collate_fn = partial(collate_batch, my_vocab=tweet_vocab)

In [26]:
emotion_labels = [
    "anger", "anticipation", "disgust", "fear", "joy", "love",
    "optimism", "pessimism", "sadness", "surprise", "trust"
]
my_config = CustomConfig(
    vocab_size=len(tweet_vocab),
    embedding_dim=300,     # Embedding dimension
    hidden_dim1=400,       # First hidden layer size
    hidden_dim2=300,       # Second hidden layer size
    hidden_dim3=200,       # Third hidden layer size
    num_labels=11,         # Number of labels for multilabel classification
    dropout_rate=0.5,      # Dropout rate for regularization
    id2label={i: emotion_labels[i] for i in range(len(emotion_labels))},
    label2id={emotion_labels[i]: i for i in range(len(emotion_labels))}
)


In [27]:
my_config

CustomConfig {
  "dropout_rate": 0.5,
  "embedding_dim": 300,
  "hidden_dim1": 400,
  "hidden_dim2": 300,
  "hidden_dim3": 200,
  "id2label": {
    "0": "anger",
    "1": "anticipation",
    "2": "disgust",
    "3": "fear",
    "4": "joy",
    "5": "love",
    "6": "optimism",
    "7": "pessimism",
    "8": "sadness",
    "9": "surprise",
    "10": "trust"
  },
  "label2id": {
    "anger": 0,
    "anticipation": 1,
    "disgust": 2,
    "fear": 3,
    "joy": 4,
    "love": 5,
    "optimism": 6,
    "pessimism": 7,
    "sadness": 8,
    "surprise": 9,
    "trust": 10
  },
  "transformers_version": "4.45.1",
  "vocab_size": 8134
}

**<font color="blue">Initialize the model</font>**

In [28]:
model = CustomMLP(config=my_config)

In [29]:
model

CustomMLP(
  (embedding_bag): EmbeddingBag(8134, 300, mode='mean')
  (lstm): LSTM(300, 400, batch_first=True, bidirectional=True)
  (fc): Sequential(
    (0): Linear(in_features=800, out_features=300, bias=True)
    (1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=300, out_features=200, bias=True)
    (5): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): LeakyReLU(negative_slope=0.01)
    (7): Dropout(p=0.5, inplace=False)
    (8): Linear(in_features=200, out_features=11, bias=True)
  )
)

**<font color="blue">Evaluation Metrics</font>**

In [30]:
import numpy as np
import evaluate

def compute_metrics(eval_pred):
    # Load F1 metric designed for multilabel classification
    f1_metric = evaluate.load("f1","multilabel")

    # Unpack logits and labels
    logits, labels = eval_pred

    # Convert logits to probabilities using sigmoid (for multilabel classification)
    probabilities = torch.sigmoid(torch.tensor(logits))
    #After trial and error with different thresholds, choosing 0.38 as the optimum threshold that gives better generalization for all classes
    # Apply a threshold of 0.38 to get binary predictions
    predictions = (probabilities > 0.38).int().numpy()

    # Compute F1 macro for multilabel classification
    f1 = f1_metric.compute(
        predictions=predictions,
        references=labels,
        average="macro"  # Macro averaging
    )

    return f1


**<font color="blue">Training Parameters(HyperTuning)</font>**

In [31]:
# Configure training parameters
from transformers import TrainingArguments, EarlyStoppingCallback
training_args = TrainingArguments(

    # Training-specific configurations
    num_train_epochs=10,
    per_device_train_batch_size=128, # Number of samples per training batch
    per_device_eval_batch_size=128, # Number of samples per validation batch
    weight_decay=0.2, # weight decay (L2 regularization)
    learning_rate=0.01, # learning rate
    optim='adamw_torch', # optimizer
    remove_unused_columns=False, # flag to retain unused columns
    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=20,  # Perform evaluation every 50 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=20,  # Save a model checkpoint every 50 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=2,  # Retain only the best and the most recent model checkpoints
    # Use 'f1' as the metric to determine the best model
    metric_for_best_model="f1",
    greater_is_better=True,  # A model is 'better' if its f1 is higher
    # Experiment logging configurations
    logging_strategy='steps',
    logging_steps=20,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name='tweet_hf_trainer',  # Experiment name for Weights & Biases
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [32]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset,
    eval_dataset = validset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)


In [33]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [34]:
# specify the project name where the experiment will be logged
%env WANDB_PROJECT = emotion-detector-hf-trainer

env: WANDB_PROJECT=emotion-detector-hf-trainer


**<font color="blue">Train the model</font>**

In [35]:
trainer.train()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: meghacnair96 (meghacnair96-the-university-of-texas-at-dallas). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,F1
20,0.751500,0.740822,0.262474
40,0.714400,0.701543,0.281134
60,0.649000,0.677696,0.333210
80,0.606500,0.643141,0.347389
100,0.601400,0.630608,0.356263
120,0.537100,0.663658,0.356426
140,0.528100,0.620503,0.379218
160,0.489300,0.643276,0.391415
180,0.481600,0.637841,0.399822
200,0.481400,0.624253,0.401757


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


TrainOutput(global_step=490, training_loss=0.4534516918415926, metrics={'train_runtime': 59.4785, 'train_samples_per_second': 1038.863, 'train_steps_per_second': 8.238, 'total_flos': 29585258375940.0, 'train_loss': 0.4534516918415926, 'epoch': 10.0})

**Based on the above observations, it may be helpful in further trials to consider early stopping or model regularization (such as increasing dropout or weight decay) to prevent overfitting after step 300. Additionally, reviewing the model’s behavior after this point to fine-tune parameters might help achieve better generalization.**

In [36]:
# After training, let us check the best checkpoint
# We need this for Inference
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 460.


In [37]:
wandb.finish()

eval/f1,▁▂▄▄▄▄▅▆▆▆▆▇▆▇▇█████████
eval/loss,█▆▄▂▂▄▁▂▂▁▃▂▆▃▃▄▅▄▅▅▇▆▇▇
eval/runtime,█▁▁▁▁▂▂▂▁▁▁▁▂▂▂▁▁▁▂▂▃▁▁▁
eval/samples_per_second,▁▇▇▇█▇▆▇██▇█▇▇▇▇██▆▆▅██▇
eval/steps_per_second,▁▇▇▇█▇▆▇██▇█▇▇▇▇██▆▆▅██▇
train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇████
train/grad_norm,█▇▄▅▄▃▃▁▂▂▄▂▄▂▄▄▃▁▃▄▂▃▂▃
train/learning_rate,██▇▇▇▆▆▆▆▅▅▅▄▄▄▃▃▃▃▂▂▂▁▁
train/loss,█▇▆▆▆▅▅▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁▁
eval/f1,0.4492


**<font color="blue">Saving the model checkpoint step for best model performace</font>**

In [38]:
#model_checkpoint' variable is constructed using the model folder path and the checkpoint step
# This step is identified as having the best model performance during training
model_checkpoint = model_folder/f'checkpoint-{best_model_checkpoint_step}'

In [39]:
# Load the pre-trained weights into the CustomMLP model from the specified checkpoint
# 'model_checkpoint' refers to the path where the model's best-performing state is saved
# This step ensures the model is initialized with weights from its most effective training state
model = model.from_pretrained(model_checkpoint, config = my_config)

In [40]:
# Create a partial function 'collate_fn' using 'collate_batch' with 'my_vocab' set to 'imdb_vocab'
# This function will be used by the Trainer to process batches of data during evaluation
collate_fn = partial(collate_batch, my_vocab=tweet_vocab)

# Configure training arguments for model evaluation
# 'output_dir' specifies where to save the results
# 'per_device_eval_batch_size' sets the batch size for evaluation, adjusted based on available GPU memory
# 'do_train = False' and 'do_eval=True' indicate that training is not performed, but evaluation is
# 'remove_unused_columns=False' ensures that all columns in the dataset are retained during evaluation
# 'report_to=[]' disables logging to external services like Weights & Biases

training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=16,
    do_train=False,
    do_eval=True,
    remove_unused_columns=False,
    report_to=[]
)

**<font color="blue">Initialise the model and training arguments</font>**

In [41]:
# Initialize the Trainer with the specified model and training arguments
# 'model' is the CustomMLP model loaded with pre-trained weights
# 'training_args' contains the configurations for evaluation, including batch sizes and output directory
# 'eval_dataset' is set to 'testset', which is the dataset used for evaluating the model
# 'data_collator' is assigned 'collate_fn', the function for processing batches of data
# 'compute_metrics' is a function that calculates evaluation metrics like accuracy and F1 score
def collate_fn_for_test(batch):
    return collate_batch(batch, tweet_vocab, is_test=True) #since testdata doesnt have labels adding a is_test Flag to run the collate_fn in test mode.
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=testset,
    data_collator=collate_fn_for_test
)


**<font color="blue">Get predictions</font>**

In [46]:
# Perform prediction
predictions_output = trainer.predict(testset)

# Extract logits from the output
logits = predictions_output.predictions

# Convert logits to probabilities (multilabel classification)
probabilities = torch.sigmoid(torch.tensor(logits))

# Apply threshold to convert probabilities to binary predictions
predictions = (probabilities > 0.38).int().numpy()

# Print or save the predictions
print(predictions)

[[0 1 0 ... 0 0 1]
 [1 0 1 ... 1 0 0]
 [0 0 1 ... 1 0 0]
 ...
 [1 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 1]
 [1 1 1 ... 0 0 0]]


**<font color="blue">Create Predictions DataFrame and store it into a CSV</font>**

In [47]:
predictionsDataFrame = pd.DataFrame(testdata['ID'], columns=['ID'])


In [48]:
# Convert predictions to a DataFrame
df = pd.DataFrame(predictions, columns=emotion_labels)

predictionsDataFrame = pd.concat([predictionsDataFrame, df], axis=1)
predictionsDataFrame

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,0,1,0,0,1,0,1,0,0,0,1
1,2018-03739,1,0,1,0,0,0,0,0,1,0,0
2,2018-00385,0,0,1,1,0,0,0,1,1,0,0
3,2018-03001,0,0,0,0,0,0,0,1,1,0,0
4,2018-01988,0,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-03848,0,1,0,0,1,0,1,0,0,0,0
3255,2018-00416,1,0,1,1,0,0,0,0,1,0,0
3256,2018-03717,1,0,1,0,0,0,0,0,0,0,0
3257,2018-03504,0,1,0,0,1,0,1,0,0,0,1


In [45]:
predictionsDataFrame.to_csv('predictions.csv', index=False)